In [1]:
%load_ext autoreload
%autoreload 2

import pprint, json, math, os, sys
import fitz
import pandas as pd
from collections import defaultdict

dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"

# dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
# fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'
sys.path.append(os.path.abspath(dir_path))

from app.fundData import *
from app.helper import Helper

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
rep_path = r'\data\output\pdf_report.xlsx'

json_path = dir_path + r'\data\output\dump.json'
pkl_path = dir_path + r'\data\output\sample.pkl'

mutual_fund = Helper.get_fund_paths(fund_path)

In [ ]:
"""WHITEOAK MAIN FILE CODE""" #Issue: Inv Obj in the right #Fund Manager Data
object = WhiteOak(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['WhiteOak Mutual Fund']

path, df = object.check_and_highlight(file_path, 6)

title = df.title.to_dict()
pages = [7, 9, 11, 13, 15, 17, 19, 20, 21, 23, 25, 26, 28, 30, 32, 34, 35]
data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_empty_dict_values(final_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
'''' 360 ONE MAIN FILE CODE''' #No issues as of now

object = ThreeSixtyOne(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['360 ONE Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [5, 6, 7, 8, 9, 10, 11, 12]
data = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
""" MAHINDRA MANULIFE MAIN CODE""" #incrase pages count

object = MahindraManu(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Mahindra Manulife Mutual Fund']

path,df = object.check_and_highlight(file_path, 7)

title = object.get_proper_fund_names(file_path,[i for i in range(1,80)])
pages = [i for i in range(8,34)]

data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""JM FUND MAIN FILE CODE""" #objective on left, sometimes date is with header

object = JMMF(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['JM Financial Mutual Fund']


path, df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37]

data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
""" INVESCO MF MAIN FILE CODE""" #Issue: two fund on same page

object = Invesco(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Invesco Mutual Fund']

path,df = object.check_and_highlight(file_path, 7)
title = df.title.to_dict()
pages = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
""" SAMCO PDF FILE MAIN CODE""" #Issues: TTER data regex rst fine !!

object = Samco(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund['Samco Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)
title = df.title.to_dict()
pages =  [3, 5, 7, 9, 11, 13, 15, 17, 18]

data = object.extract_clipped_data(file_path,pages, title)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)


In [ ]:
""" TATA FILE MAIN CODE """
object = Tata(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund["Tata Mutual Fund"]

path, df = object.check_and_highlight(file_path, 10)

title = df.title.to_dict()
pages = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]

data = object.extract_clipped_data(file_path,pages, title)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = Helper.drop_empty_dict_values(extracted_text)
final_text = object.refine_extracted_data(final_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
""" FRANKLIN TEMPLETON FILE MAIN CODE"""

object = FranklinTempleton(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund["Franklin Templeton Mutual Fund"]

path, df = object.check_and_highlight(file_path, 6)
title = df.title.to_dict()
pages = [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

data = object.extract_clipped_data(file_path,pages, title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""BANDHAN MF FILE MAIN CODE"""

object = Bandhan(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund["Bandhan Mutual Fund"]

path, df = object.check_and_highlight(file_path,6)
pages = [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]
title = df.title.to_dict()

data = object.extract_clipped_data(file_path,pages, title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""KOTAK FUND MAIN CODE""" #Issue: 1 or more fund on same page, # title overlap the content
object = Kotak(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Kotak Mahindra Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 36, 37, 38, 40, 41, 43, 44, 46, 48, 49, 50, 52, 54, 55, 56, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]
data = object.extract_clipped_data(file_path,pages,title)#used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_selected_dict_values(final_text, ['ideal_investments_horizon','before','idcw_frequency','available_plansoptions'])
final_text = Helper.drop_keys_by_regex(final_text,[r'^folio'])
final_text = Helper.drop_empty_dict_values(final_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""ZERODHA MAIN CODE""" 

object = Zerodha(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Zerodha Mutual Fund']

path,df= object.check_and_highlight(file_path,7)
title = df.title.to_dict()
pages = [3, 5, 7, 9, 11, 13, 15]

#left
data = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data)
nested_data_left, matrix = object.create_nested_dict(clean_data,30.0, 14.0)

#right
data = object.extract_data_relative_line(file_path,pages,"right",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data)
nested_data_right, matrix = object.create_nested_dict(clean_data,30.0, 14.0)

nested_data = Helper.merge_nested_dicts(nested_data_left, nested_data_right)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)


In [ ]:
"""NAVI MAIN FILE CODE""" #Issue: Left and right

object = NAVI(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Navi Mutual Fund']


path,df = object.check_and_highlight(file_path,5)
title = df.title.to_dict()
pages = [2, 6, 8, 10, 13, 16, 18, 20, 22]

#left
data = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data)
nested_data_left, matrix = object.create_nested_dict(clean_data,30.0, 9.0)

#right
data = object.extract_data_relative_line(file_path,pages,"right",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data)
nested_data_right, matrix = object.create_nested_dict(clean_data,30.0, 9.0)

nested_data = Helper.merge_nested_dicts(nested_data_left, nested_data_right)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_selected_dict_values(final_text,['industry_allocation','equity_','this_product_is_suitable_for_investors_who_are_seeking'])
Helper.quick_json_dump(final_text, json_path)

In [ ]:
""" BARODA BNP MAIN FILE CODE""" #Issue: Header funds #scheme details

object = BarodaBNP(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Baroda BNP Paribas Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)


pages = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
title = object.get_proper_fund_names(file_path, pages, (0,0,210,75))

data = object.extract_data_relative_line(file_path,pages,"left", title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_empty_dict_values(final_text)
final_text = Helper.drop_keys_by_regex(final_text,[r"^(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)",r'^key_statistics',r'^the_risk_free_rate_of_retu'])
Helper.quick_json_dump(final_text, json_path)

In [ ]:
""" MOTILAL OSWAL MAIN CODE FILE""" #Fund Manager Regex

object = MotilalOswal(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Motilal Oswal Mutual Fund']

path,df= object.check_and_highlight(file_path, 7)

pages = [i for i in range(4,21)]
title = df.title.to_dict()
data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""HELIOS MF FILE MAIN CODE""" #fund manager 

object = Helios(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund['Helios Mutual Fund']

path, df = object.check_and_highlight(file_path,6)
title = df.title.to_dict()
pages =  [2, 4, 6, 8, 10]

data = object.extract_data_relative_line(file_path,pages ,'left', title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)


In [ ]:
final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_keys_by_regex(final_text,[r'^(An open|Large).*'])
Helper.quick_json_dump(final_text,json_path)

In [ ]:
grand_head = []
for content in final_text.values():
    for key in content.keys():
        grand_head.append(key)

grand_head = set(grand_head)

grand_head

In [ ]:
"""EDELWEISS MP FILE MAIN CODE""" #Issues: objective in seperate dict, scheme launch date issue

object = Edelweiss(dir_path, dry_path, fin_path, rep_path)
file_path = mutual_fund['Edelweiss Mutual Fund']


path, df = object.check_and_highlight(file_path, 6)

pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]

title,objectives = object.get_proper_fund_names(file_path,pages)

data = object.extract_data_relative_line(file_path,pages ,'right',title ) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data)

final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

In [ ]:
"""NJMF MAIN CODE FILE"""

object = NJMF(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['NJ Mutual Fund']

# path,df = object.check_and_highlight(file_path, 7)
pages = [1, 3, 5, 7, 9]
title = df.title.to_dict()

data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""SUNDARAM MAIN FILE CODE"""

object = Sundaram(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Sundaram Mutual Fund']

# path,df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
data = object.extract_data_relative_line(file_path,pages,"left",title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)

final_text = object.refine_extracted_data(extracted_text)
#below functions are optional to run
final_text = Helper.drop_selected_dict_values(final_text,['performance_10,000_invested_value_of_10,000-a-month_sip','market_capital','~since_inception','last_5_years','last_3_years','last_1_year','period'])
final_text = Helper.drop_empty_dict_values(final_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
""" ITI MAIN FILE CODE""" #Issues: more data cleaning

object = ITI(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['ITI Mutual Fund']

# path, df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)

final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_selected_dict_values(final_text,['this_product_is_suitable'])
final_text = Helper.drop_empty_dict_values(final_text)
Helper.quick_json_dump(final_text, json_path)

In [ ]:
"""MIRAE MAIN FILE CODE"""

object = MIRAE(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Mirae Asset Mutual Fund']

# path, df = object.check_and_highlight(file_path, 7)
pages = [25, 26, 27, 28, 29, 30, 31, 32, 33, 34,35, 36, 37, 38, 39, 40, 41,42,43,44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,61,62,63,64, 65, 66, 67, 68]
title = object.get_proper_fund_names(file_path,pages)
data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text, json_path)


In [ ]:
""" BANK OF INDIA""" #Issues: Fund Manger extraction
object = BankOfIndia(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Bank of India Mutual Fund']

path, df= object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

#left
data_l = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data_l = object.extract_span_data(data_l,[])
clean_data = object.process_text_data(data_l)
nested_data_left, matrix = object.create_nested_dict(clean_data,30.0, 10.0)

#right
data_r = object.extract_data_relative_line(file_path,pages,"right",title) #used line here
data_r = object.extract_span_data(data_r,[])
clean_data = object.process_text_data(data_r)
nested_data_right, matrix = object.create_nested_dict(clean_data,30.0, 10.0)

nested_data = {key:{**nested_data_left[key],**nested_data_right[key] }for key in nested_data_left.keys()}
extracted_text = object.get_generated_content(nested_data)

final_text = object.refine_extracted_data(extracted_text)
remove = ['certificate_of_deposit','treasury_bill','who_are_seeking*:','who_should_invest','equity_holdings','this_product_is_suitable_for_investors','december_31,_2024']
final_clean_text = Helper.drop_selected_dict_values(final_text, remove)
final_clean_text = Helper.drop_keys_by_regex(final_clean_text, [r'^total'])
final_clean_text = Helper.drop_empty_dict_values(final_clean_text)
Helper.quick_json_dump(final_clean_text, json_path)

In [ ]:
"""TAURUS MAIN FILE CODE""" #Issues: Nothing Yet

object = Taurus(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Taurus Mutual Fund']

# path, df= object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [13, 14, 15, 16, 17, 19, 20]
data = object.extract_data_relative_line(file_path,pages,"left",title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

In [ ]:
"""TRUST MAIN FILE CODE""" #Issue: Clean data more, nested dict unload Invest Obj in right #Names coming as headers fund managers

object = Trust(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Trust Mutual Fund']

# path, df= object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [6, 7, 8, 9, 10, 11, 12, 13]
data = object.extract_clipped_data(file_path,pages,title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

In [ ]:
"""CANARA MUTUAL FUND""" #Issues: very complicated ""HHHHHHHHHHHHHHHHHHHHHHHH

object = Canara(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Canara Robeco Mutual Fund']

# path, df = object.check_and_highlight(file_path, 7)

pages = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
title = df.title.to_dict()

#segment 1
data_a = object.extract_clipped_data(file_path,pages,title) #used line here
data_a = object.extract_span_data(data_a,[])
clean_data_a = object.process_text_data(data_a)
nested_data_a, matrix = object.create_nested_dict(clean_data_a,30.0, 10.0)

#segment 2
data_b = object.extract_clipped_data(file_path, pages, title,[(220,115,400,812)])
data_b = object.extract_span_data(data_b,[])
clean_data_b = object.process_text_data(data_b)
nested_data_b, matrix = object.create_nested_dict(clean_data_b,30.0, 10.0)


nested_data = {key:{**nested_data_a[key],**nested_data_b[key] }for key in nested_data_b.keys()}
final_text = Helper.merge_key_values(extracted_text,'fund_information','before')
final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_selected_dict_values(final_text,['portfolio','product_positioning'])
Helper.quick_json_dump(final_text,json_path)
extracted_text = object.get_generated_content(nested_data)

In [ ]:
""" BAJAJ FINSERV MAIN FILE CODE""" #major issue of lines #minor of dropping keys of final dict

object = BajajFinServ(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Bajaj finserv Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)

pages = [15, 17, 19, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33]
title = df.title.to_dict()

#segmant 1 left
data_l = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data_l = object.extract_span_data(data_l,[])
clean_data = object.process_text_data(data_l)
nested_data_left, matrix = object.create_nested_dict(clean_data,30.0, 14.0)

#segment 2
data_a = object.extract_clipped_data(file_path,pages,title,[(180,45,360,300)]) #used line here
data_a = object.extract_span_data(data_a,[])
clean_data_a = object.process_text_data(data_a)
nested_data_a, matrix = object.create_nested_dict(clean_data_a,30.0, 10.0)

#segment 3
data_b = object.extract_clipped_data(file_path, pages, title,[(360,45,580,300)])
data_b = object.extract_span_data(data_b,[])
clean_data_b = object.process_text_data(data_b)
nested_data_b, matrix = object.create_nested_dict(clean_data_b,30.0, 10.0)


nested_data = Helper.merge_nested_dicts(nested_data_left, nested_data_a, nested_data_b)
extracted_text = object.get_generated_content(nested_data)

In [ ]:
"""UTI MAIN FILE CODE"""

object = UTI(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['UTI Mutual Fund']

path,df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 41, 43, 45, 46, 48, 50, 52, 54, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77]
data = object.extract_data_relative_line(file_path,pages,"left",title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
final_text = Helper.drop_selected_dict_values(final_text,['high/low_nav_in_the_month','plans/options','porolio_details','market_capital'])
Helper.quick_json_dump(final_text,json_path)

In [ ]:
"""NIPPON MUTUAL FUND"""

object = Nippon(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Nippon India Mutual Fund']

path,df = object.check_and_highlight(file_path, 7)
pages = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92]
title = df.title.to_dict()
data = object.extract_clipped_data(file_path,pages,title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

In [ ]:
"""QUANTUM MAIN FILE CODE""" #ISSUES: Clean data further

object = Quantum(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund["Quantum Mutual Fund"]

# path,df = object.check_and_highlight(file_path, 7)

title = df.title.to_dict()
pages = [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 28, 29, 30]
data = object.extract_clipped_data(file_path,pages,title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 8.0)
extracted_text = object.get_generated_content(nested_data)

In [ ]:
"""UNION MUTUAL FUND""" #Issues: Clean Further

object = Union(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Union Mutual Fund']

# path,df = object.check_and_highlight(file_path, 7)
pages = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,]
title = df.title.apply(lambda x: "Union "+ x if x !="" else x).to_dict()
data = object.extract_clipped_data(file_path,pages,title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data)
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

FILES TO BE CLEANED ARE BELOW , DATA ABLE TO EXTRACTED 

In [ ]:
"""QUANT MAIN CODE FILE"""

object = QuantMF(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Quant Mutual Fund']

path,df = object.check_and_highlight(file_path, 7)
title = df.title.to_dict()
pages = [12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 61, 63, 65]
# data = object.extract_data_relative_line(file_path,pages,"left",title) #used clip here
# data = object.extract_span_data(data,[])
# clean_data = object.process_text_data(data) 
# nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
# extracted_text = object.get_generated_content(nested_data)

""""HSBC MAIN FILE CODE""" #Issue: Some pages have 2 or more fund data on same page
#some data is present on right side as well so define two lines
#objective is to be exracted seperately

object = HSBC(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['HSBC Mutual Fund']

path, df = object.check_and_highlight(file_path, 7)
title = df.title.to_dict()

pages = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
data = object.extract_clipped_data(file_path, pages, title) #used line here
data = object.extract_span_data(data,[])

clean_data = object.process_text_data(data) 
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)

In [13]:
"""PGIM MUTUAL FUND"""

object = PGIM(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['PGIM India Mutual Fund']

# path,df = object.check_and_highlight(file_path, 7)
pages = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 29, 30, 31, 32, 33, 34]
title = df.title.apply(lambda x: "PGIM "+x if not x == "" else x).to_dict()
data = object.extract_clipped_data(file_path,pages,title) #used line here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 

In [ ]:
nested_data, matrix = object.create_nested_dict(clean_data,30.0, 10.0)
extracted_text = object.get_generated_content(nested_data)

In [ ]:
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

In [ ]:
"""DSP MAIN FILE CODE""" #Issue: highlight count must be lower
#Issue: data on both side ends so issue coming

object = DSP(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['DSP Mutual Fund']

# path, df = object.check_and_highlight(file_path, 5)

title = df.title.to_dict()
pages = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]

#segment 1
data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data1, matrix = object.create_nested_dict(clean_data,30.0, 10.0)

#segment2
data = object.extract_clipped_data(file_path,pages,title,[(480,5,596,812)]) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data2, matrix = object.create_nested_dict(clean_data,30.0, 10.0)

nested_data = Helper.merge_nested_dicts(nested_data1, nested_data2)
extracted_text = object.get_generated_content(nested_data)

In [ ]:
final_text = object.refine_extracted_data(extracted_text)
Helper.quick_json_dump(final_text,json_path)

In [ ]:
"""ADITYA BIRLA FILE CODE""" #Issue: highlight count must be lower
#Issue: data on both side ends so issue coming

object = AdityaBirla(dir_path,dry_path,fin_path, rep_path)
file_path = mutual_fund['Aditya Birla Sun Life Mutual Fund']

# path, df = object.check_and_highlight(file_path, 16)

title = df.title.to_dict()
pages = [12, 14, 16, 18, 20, 22, 24, 26, 27, 28, 30, 32, 34, 35, 36, 38, 40, 44, 47, 49, 53, 55, 57, 61, 63, 66, 68, 69, 72, 73, 74, 75, 76, 78, 80, 81, 85, 95, 102, 104, 108, 110, 112, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 141, 143, 147, 149, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 170, 171, 172, 173, 174]
#segment 1
data = object.extract_clipped_data(file_path,pages,title) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data1, matrix = object.create_nested_dict(clean_data,30.0, 10.0)

#segment2
data = object.extract_clipped_data(file_path,pages,title,[(200, 50, 380, 812)]) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data2, matrix = object.create_nested_dict(clean_data,30.0, 10.0)

#segment3
data = object.extract_clipped_data(file_path,pages,title,[(380, 50, 580, 812)]) #used clip here
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data) 
nested_data3, matrix = object.create_nested_dict(clean_data,30.0, 10.0)

nested_data = Helper.merge_nested_dicts(nested_data1, nested_data2,nested_data3)
extracted_text = object.get_generated_content(nested_data)

In [ ]:
final_text = object.refine_extracted_data(extracted_text)
# final_text = Helper.drop_empty_dict_values(final_text)
Helper.quick_json_dump(final_text,json_path)